In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.utils import resample

# Data

In [4]:
np.random.seed(2)

dataset_size = ["small", "medium", "large", "beta"][1]

dataset_info = {
    "small": {
        "dataset_name": "wine",
        "class_name": "Class",
        "drop_fields": []
    },
    "medium": {
        "dataset_name": "breast-cancer-wisconsin",
        "class_name": "Class",
        "drop_fields": ["Sample code number"]
    },
    "large": {
        "dataset_name": "seismic-bumps",
        "class_name": "class",
        "drop_fields": []
    },
    "beta": {
        "dataset_name": "Acoustic_Extinguisher_Fire_Dataset",
        "class_name": "Class",
        "drop_fields": []
    }
}

dataset_name = dataset_info[dataset_size]["dataset_name"]
class_name = dataset_info[dataset_size]["class_name"]
drop_fields = dataset_info[dataset_size]["drop_fields"]

df = pd.read_csv('../data/' + dataset_name + ".csv")
df = df.drop(drop_fields, axis=1)
df = df.iloc[np.random.permutation(len(df))]

if dataset_name == "breast-cancer-wisconsin":
    df[class_name].replace({2: 0, 4: 1}, inplace=True)
    
if dataset_name == "ThoracicSurgery":
    df[class_name].replace({'T': 1, 'F': 0}, inplace=True)

n_cut = int(0.8*len(df))
df_trn = df[:n_cut]
df_tst = df[n_cut:]

if dataset_name == "ThoracicSurgery":
    df_1 = df_trn[df_trn[class_name] == 1]
    df_0 = df_trn[df_trn[class_name] != 1]  
    df_1_upsampled = resample(df_1, random_state=2, n_samples=len(df_0), replace=True)

    df_upsampled = pd.concat([df_1_upsampled, df_0], ignore_index=True)
    df_upsampled = df_upsampled.iloc[np.random.permutation(df_upsampled.index)]
    
    df_0_downsampled = resample(df_0, random_state=20, n_samples=len(df_1), replace=False)
    df_downsampled = pd.concat([df_0_downsampled, df_1], ignore_index=True)
    df_downsampled = df_downsampled.iloc[np.random.permutation(df_downsampled.index)]
    
    # df_trn = df_upsampled
    df_trn = df_downsampled

X_trn = df_trn.drop(class_name, axis=1)
y_trn = df_trn[class_name]

X_tst = df_tst.drop(class_name, axis=1)
y_tst = df_tst[class_name]

# Random Forest

In [3]:
from RandomForest_df import RandomForest_df

M = X_trn.shape[1]
CV_dict_params = {'NT': [1, 10, 25, 50, 75, 100],
                  'F': sorted(list(set([1, 3, int(np.log2(M + 1)), int(np.sqrt(M))])))
                  }

best_F, best_NT, best_score = None, None, -1
all_metrics_CV = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        all_scores = np.zeros(5)
        N = len(X_trn)
        for run in range(5):
            
            ind_tst_ = np.full(len(X_trn), False)
            ind_tst_[int(N*run/5):int(N*(run+1)/5)] = True
            
            X_trn_, y_trn_ = X_trn[ind_tst_], y_trn[ind_tst_]
            X_tst_, y_tst_ = X_trn[~ind_tst_], y_trn[~ind_tst_]
            
            clf = RandomForest_df(NT=NT_, F=F_)
            clf.fit(X_trn_, y_trn_)
            all_scores[run] = clf.score(X_tst_, y_tst_)
            
        score_ = all_scores.mean()
        all_metrics_CV.append(score_)
        
        if score_ > best_score:
            best_F, best_NT, best_score = F_, NT_, score_
        print(f'(F, NT) = {(F_, NT_)} \t--> \t F1-Score = {round(score_, 3)}')
        
all_metrics_CV = np.array(all_metrics_CV)


(F, NT) = (1, 1) 	--> 	 F1-Score = 0.89
(F, NT) = (1, 10) 	--> 	 F1-Score = 0.931
(F, NT) = (1, 25) 	--> 	 F1-Score = 0.944
(F, NT) = (1, 50) 	--> 	 F1-Score = 0.948
(F, NT) = (1, 75) 	--> 	 F1-Score = 0.946
(F, NT) = (1, 100) 	--> 	 F1-Score = 0.95
(F, NT) = (3, 1) 	--> 	 F1-Score = 0.867
(F, NT) = (3, 10) 	--> 	 F1-Score = 0.92
(F, NT) = (3, 25) 	--> 	 F1-Score = 0.934
(F, NT) = (3, 50) 	--> 	 F1-Score = 0.932
(F, NT) = (3, 75) 	--> 	 F1-Score = 0.937
(F, NT) = (3, 100) 	--> 	 F1-Score = 0.938


In [4]:
print(f'Best Parameters (F, NT): {(best_F, best_NT)}')
best_RF = RandomForest_df(NT=best_NT, F=best_F)
best_RF.fit(X_trn, y_trn, verbose=3)

Best Parameters (F, NT): (1, 100)
Feature importance: Index(['Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Bare Nuclei', 'Bland Chromatin',
       'Single Epithelial Cell Size', 'Marginal Adhesion', 'Normal Nucleoli',
       'Mitoses'],
      dtype='object')


RandomForest_df()

In [5]:
y_trn_hat = best_RF.predict(X_trn)
y_tst_hat = best_RF.predict(X_tst)

In [6]:
print(f'Accuracy (test): {round(accuracy_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Precision (test): {round(precision_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Recall (test): {round(recall_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'F1 Score (test): {round(f1_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print('-'*15)
print(f'Accuracy (train): {round(accuracy_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Precision (train): {round(precision_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Recall (train): {round(recall_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'F1 Score (train): {round(f1_score(y_trn.to_numpy(), y_trn_hat), 3)}')


Accuracy (test): 0.964
Precision (test): 0.942
Recall (test): 0.961
F1 Score (test): 0.951
---------------
Accuracy (train): 0.998
Precision (train): 1.0
Recall (train): 0.995
F1 Score (train): 0.997


In [7]:
all_importances = []
all_metrics = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        clf = RandomForest_df(NT=NT_, F=F_)
        print(f'(F, NT) = {(F_, NT_)}')
        clf.fit(X_trn, y_trn, verbose=3)
        all_importances.append(np.flip(np.argsort(clf.importance)))
        y_tst_hat = clf.predict(X_tst)     
        
        acc = accuracy_score(y_tst.to_numpy(), y_tst_hat)
        prec = precision_score(y_tst.to_numpy(), y_tst_hat)
        rec = recall_score(y_tst.to_numpy(), y_tst_hat)
        f1_ = f1_score(y_tst.to_numpy(), y_tst_hat)
           
        print(f'Accuracy (test): {round(acc, 3)}')
        print(f'Precision (test): {round(prec, 3)}')
        print(f'Recall (test): {round(rec, 3)}')
        print(f'F1 Score (test): {round(f1_, 3)}')
        print('-'*15)
        
        all_metrics.append([acc, prec, rec, f1_])
        
all_importances = np.array(all_importances)
all_metrics = np.array(all_metrics)

(F, NT) = (1, 1)
Feature importance: Index(['Bare Nuclei', 'Clump Thickness', 'Uniformity of Cell Size',
       'Normal Nucleoli', 'Bland Chromatin', 'Mitoses',
       'Uniformity of Cell Shape', 'Single Epithelial Cell Size',
       'Marginal Adhesion'],
      dtype='object')
Accuracy (test): 0.879
Precision (test): 0.905
Recall (test): 0.745
F1 Score (test): 0.817
---------------
(F, NT) = (1, 10)
Feature importance: Index(['Uniformity of Cell Shape', 'Uniformity of Cell Size',
       'Clump Thickness', 'Normal Nucleoli', 'Single Epithelial Cell Size',
       'Bare Nuclei', 'Marginal Adhesion', 'Mitoses', 'Bland Chromatin'],
      dtype='object')
Accuracy (test): 0.964
Precision (test): 0.942
Recall (test): 0.961
F1 Score (test): 0.951
---------------
(F, NT) = (1, 25)
Feature importance: Index(['Clump Thickness', 'Uniformity of Cell Shape',
       'Uniformity of Cell Size', 'Bare Nuclei', 'Single Epithelial Cell Size',
       'Bland Chromatin', 'Marginal Adhesion', 'Normal Nucleoli'

In [8]:
all_metrics_CV

array([0.8899331 , 0.93065862, 0.94352232, 0.94809554, 0.94607281,
       0.94956512, 0.86654785, 0.9195238 , 0.93358203, 0.93242836,
       0.93675837, 0.93761313])

In [9]:
all_metrics_CV

array([0.8899331 , 0.93065862, 0.94352232, 0.94809554, 0.94607281,
       0.94956512, 0.86654785, 0.9195238 , 0.93358203, 0.93242836,
       0.93675837, 0.93761313])

In [10]:
print(all_metrics.T)

[[0.87857143 0.96428571 0.97142857 0.97857143 0.97142857 0.96428571
  0.9        0.95       0.94285714 0.95       0.95714286 0.96428571]
 [0.9047619  0.94230769 0.92727273 0.94444444 0.94339623 0.94230769
  0.91111111 0.94       0.92156863 0.94       0.94117647 0.94230769]
 [0.74509804 0.96078431 1.         1.         0.98039216 0.96078431
  0.80392157 0.92156863 0.92156863 0.92156863 0.94117647 0.96078431]
 [0.8172043  0.95145631 0.96226415 0.97142857 0.96153846 0.95145631
  0.85416667 0.93069307 0.92156863 0.93069307 0.94117647 0.95145631]]


In [11]:
print(all_metrics_CV.T)

[0.8899331  0.93065862 0.94352232 0.94809554 0.94607281 0.94956512
 0.86654785 0.9195238  0.93358203 0.93242836 0.93675837 0.93761313]


In [12]:
from utils.print_latex import print_table

print("IMPORTANCES")
print_table(all_importances.T)
print("-"*15)

print("ACC - PRECISION - RECALL - F1 (TEST)")
print_table(all_metrics.T)
print("-"*15)

print("F1 (CV)")
print_table(all_metrics_CV)
print("-"*15)


IMPORTANCES
\textbf{\#1} & & 5 & 2 & 0 & 0 & 0 & 0 & 5 & 5 & 5 & 5 & 5 & 5\\
\hline
\textbf{\#2} & & 0 & 1 & 2 & 2 & 1 & 1 & 3 & 1 & 1 & 6 & 0 & 0\\
\hline
\textbf{\#3} & & 1 & 0 & 1 & 1 & 5 & 2 & 6 & 2 & 6 & 1 & 1 & 6\\
\hline
\textbf{\#4} & & 7 & 7 & 5 & 5 & 2 & 5 & 4 & 6 & 0 & 0 & 6 & 1\\
\hline
\textbf{\#5} & & 6 & 4 & 4 & 4 & 3 & 6 & 2 & 0 & 7 & 2 & 2 & 2\\
\hline
\textbf{\#6} & & 8 & 5 & 6 & 7 & 4 & 4 & 1 & 3 & 2 & 3 & 4 & 3\\
\hline
\textbf{\#7} & & 2 & 3 & 3 & 3 & 6 & 3 & 0 & 7 & 3 & 4 & 3 & 4\\
\hline
\textbf{\#8} & & 4 & 8 & 7 & 6 & 7 & 7 & 7 & 4 & 4 & 7 & 7 & 7\\
\hline
\textbf{\#9} & & 3 & 6 & 8 & 8 & 8 & 8 & 8 & 8 & 8 & 8 & 8 & 8\\
\hline

---------------
ACC - PRECISION - RECALL - F1 (TEST)
\textbf{\#1} & & 0.879 & 0.964 & 0.971 & 0.979 & 0.971 & 0.964 & 0.9 & 0.95 & 0.943 & 0.95 & 0.957 & 0.964\\
\hline
\textbf{\#2} & & 0.905 & 0.942 & 0.927 & 0.944 & 0.943 & 0.942 & 0.911 & 0.94 & 0.922 & 0.94 & 0.941 & 0.942\\
\hline
\textbf{\#3} & & 0.745 & 0.961 & 1.0 & 1.0 & 0.98 & 

In [13]:
from sys import modules
del modules["utils.print_latex"]

# Decision Tree

In [14]:
from DecisionForest_df import DecisionForest_df

M = X_trn.shape[1]
CV_dict_params = {'NT': [1, 10, 25, 50, 75, 100],
                  'F': sorted(list(set([int(M/4), int(M/2), int(3*M/4)]))) + [-1]
                  }

best_F, best_NT, best_score = None, None, -1
all_metrics_CV = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        all_scores = np.zeros(5)
        N = len(X_trn)
        for run in range(5):
            
            ind_tst_ = np.full(len(X_trn), False)
            ind_tst_[int(N*run/5):int(N*(run+1)/5)] = True
            
            X_trn_, y_trn_ = X_trn[ind_tst_], y_trn[ind_tst_]
            X_tst_, y_tst_ = X_trn[~ind_tst_], y_trn[~ind_tst_]
            
            clf = DecisionForest_df(NT=NT_, F=F_)
            clf.fit(X_trn_, y_trn_)
            all_scores[run] = clf.score(X_tst_, y_tst_)
            
        score_ = all_scores.mean()
        all_metrics_CV.append(score_)
        if score_ > best_score:
            best_F, best_NT, best_score = F_, NT_, score_
        print(f'(F, NT) = {(F_, NT_)} \t--> \t F1-Score = {round(score_, 3)}')
        
all_metrics_CV = np.array(all_metrics_CV)


(F, NT) = (2, 1) 	--> 	 F1-Score = 0.881
(F, NT) = (2, 10) 	--> 	 F1-Score = 0.924
(F, NT) = (2, 25) 	--> 	 F1-Score = 0.937
(F, NT) = (2, 50) 	--> 	 F1-Score = 0.939
(F, NT) = (2, 75) 	--> 	 F1-Score = 0.944
(F, NT) = (2, 100) 	--> 	 F1-Score = 0.948
(F, NT) = (4, 1) 	--> 	 F1-Score = 0.865
(F, NT) = (4, 10) 	--> 	 F1-Score = 0.913
(F, NT) = (4, 25) 	--> 	 F1-Score = 0.932
(F, NT) = (4, 50) 	--> 	 F1-Score = 0.937
(F, NT) = (4, 75) 	--> 	 F1-Score = 0.942
(F, NT) = (4, 100) 	--> 	 F1-Score = 0.942
(F, NT) = (6, 1) 	--> 	 F1-Score = 0.873
(F, NT) = (6, 10) 	--> 	 F1-Score = 0.904
(F, NT) = (6, 25) 	--> 	 F1-Score = 0.92
(F, NT) = (6, 50) 	--> 	 F1-Score = 0.912
(F, NT) = (6, 75) 	--> 	 F1-Score = 0.916
(F, NT) = (6, 100) 	--> 	 F1-Score = 0.915
(F, NT) = (-1, 1) 	--> 	 F1-Score = 0.872
(F, NT) = (-1, 10) 	--> 	 F1-Score = 0.894
(F, NT) = (-1, 25) 	--> 	 F1-Score = 0.914
(F, NT) = (-1, 50) 	--> 	 F1-Score = 0.916
(F, NT) = (-1, 75) 	--> 	 F1-Score = 0.916
(F, NT) = (-1, 100) 	--> 	 F1-S

In [15]:
print(f'Best Parameters (F, NT): {(best_F, best_NT)}')
best_DF = DecisionForest_df(NT=best_NT, F=best_F)
best_DF.fit(X_trn, y_trn, verbose=3)
print(np.flip(np.argsort(best_DF.importance)))

Best Parameters (F, NT): (2, 100)
Feature importance: Index(['Uniformity of Cell Shape', 'Normal Nucleoli',
       'Single Epithelial Cell Size', 'Uniformity of Cell Size',
       'Bland Chromatin', 'Marginal Adhesion', 'Clump Thickness', 'Mitoses',
       'Bare Nuclei'],
      dtype='object')
[2 7 4 1 6 3 0 8 5]


In [16]:
y_trn_hat = best_DF.predict(X_trn)
y_tst_hat = best_DF.predict(X_tst)

In [17]:
print(f'Accuracy (test): {round(accuracy_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Precision (test): {round(precision_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Recall (test): {round(recall_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'F1 Score (test): {round(f1_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print('-'*15)
print(f'Accuracy (train): {round(accuracy_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Precision (train): {round(precision_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Recall (train): {round(recall_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'F1 Score (train): {round(f1_score(y_trn.to_numpy(), y_trn_hat), 3)}')


Accuracy (test): 0.964
Precision (test): 0.942
Recall (test): 0.961
F1 Score (test): 0.951
---------------
Accuracy (train): 0.97
Precision (train): 0.948
Recall (train): 0.963
F1 Score (train): 0.956


In [18]:
all_importances = []
all_metrics = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        clf = DecisionForest_df(NT=NT_, F=F_)
        print(f'(F, NT) = {(F_, NT_)}')
        clf.fit(X_trn, y_trn, verbose=3)
        y_tst_hat = clf.predict(X_tst)   
        
        all_importances.append(np.flip(np.argsort(clf.importance)))     
        
        acc = accuracy_score(y_tst.to_numpy(), y_tst_hat)
        prec = precision_score(y_tst.to_numpy(), y_tst_hat)
        rec = recall_score(y_tst.to_numpy(), y_tst_hat)
        f1_ = f1_score(y_tst.to_numpy(), y_tst_hat)
           
        print(f'Accuracy (test): {round(acc, 3)}')
        print(f'Precision (test): {round(prec, 3)}')
        print(f'Recall (test): {round(rec, 3)}')
        print(f'F1 Score (test): {round(f1_, 3)}')
        print('-'*15)
        
        all_metrics.append([acc, prec, rec, f1_])
        
all_importances = np.array(all_importances)
all_metrics = np.array(all_metrics)

(F, NT) = (2, 1)
Feature importance: Index(['Uniformity of Cell Shape', 'Normal Nucleoli', 'Mitoses',
       'Bland Chromatin', 'Bare Nuclei', 'Single Epithelial Cell Size',
       'Marginal Adhesion', 'Uniformity of Cell Size', 'Clump Thickness'],
      dtype='object')
Accuracy (test): 0.95
Precision (test): 0.923
Recall (test): 0.941
F1 Score (test): 0.932
---------------
(F, NT) = (2, 10)
Feature importance: Index(['Uniformity of Cell Shape', 'Normal Nucleoli',
       'Single Epithelial Cell Size', 'Uniformity of Cell Size', 'Mitoses',
       'Bland Chromatin', 'Bare Nuclei', 'Marginal Adhesion',
       'Clump Thickness'],
      dtype='object')
Accuracy (test): 0.964
Precision (test): 0.942
Recall (test): 0.961
F1 Score (test): 0.951
---------------
(F, NT) = (2, 25)
Feature importance: Index(['Uniformity of Cell Shape', 'Normal Nucleoli',
       'Single Epithelial Cell Size', 'Mitoses', 'Marginal Adhesion',
       'Uniformity of Cell Size', 'Bland Chromatin', 'Bare Nuclei',
       

In [19]:
print(all_metrics.T)


[[0.95       0.96428571 0.96428571 0.96428571 0.96428571 0.96428571
  0.95       0.96428571 0.96428571 0.95714286 0.95714286 0.95714286
  0.95714286 0.94285714 0.95714286 0.95714286 0.96428571 0.95714286
  0.92142857 0.95       0.95       0.95714286 0.95714286 0.95      ]
 [0.92307692 0.94230769 0.94230769 0.94230769 0.94230769 0.94230769
  0.94       0.94230769 0.94230769 0.94117647 0.94117647 0.94117647
  0.94117647 0.93877551 0.94117647 0.94117647 0.94230769 0.94117647
  0.91666667 0.94       0.92307692 0.94117647 0.94117647 0.94      ]
 [0.94117647 0.96078431 0.96078431 0.96078431 0.96078431 0.96078431
  0.92156863 0.96078431 0.96078431 0.94117647 0.94117647 0.94117647
  0.94117647 0.90196078 0.94117647 0.94117647 0.96078431 0.94117647
  0.8627451  0.92156863 0.94117647 0.94117647 0.94117647 0.92156863]
 [0.93203883 0.95145631 0.95145631 0.95145631 0.95145631 0.95145631
  0.93069307 0.95145631 0.95145631 0.94117647 0.94117647 0.94117647
  0.94117647 0.92       0.94117647 0.94117647

In [20]:
print(all_metrics_CV.T)

[0.88112134 0.92389866 0.93652462 0.93916992 0.94355389 0.94753569
 0.86548238 0.91333236 0.93222102 0.93707257 0.94195359 0.94188489
 0.87313452 0.90412096 0.92021968 0.91225964 0.91581251 0.91481316
 0.87227707 0.89398921 0.91421847 0.91553608 0.91609245 0.91827973]


In [21]:
from utils.print_latex import print_table

print("IMPORTANCES")
print_table(all_importances.T)
print("-"*15)

print("ACC - PRECISION - RECALL - F1 (TEST)")
print_table(all_metrics.T)
print("-"*15)

print("F1 (CV)")
print_table(all_metrics_CV)
print("-"*15)


IMPORTANCES
\textbf{\#1} & & 2 & 2 & 2 & 2 & 2 & 2 & 4 & 7 & 7 & 7 & 7 & 7 & 7 & 0 & 2 & 2 & 1 & 5 & 5 & 5 & 5 & 5 & 5 & 5\\
\hline
\textbf{\#2} & & 7 & 7 & 7 & 7 & 4 & 7 & 7 & 4 & 2 & 2 & 1 & 1 & 4 & 2 & 7 & 1 & 2 & 1 & 1 & 1 & 2 & 1 & 1 & 1\\
\hline
\textbf{\#3} & & 8 & 4 & 4 & 1 & 1 & 4 & 2 & 2 & 4 & 1 & 4 & 2 & 1 & 7 & 0 & 5 & 5 & 0 & 2 & 2 & 1 & 2 & 2 & 0\\
\hline
\textbf{\#4} & & 6 & 1 & 8 & 6 & 7 & 1 & 1 & 6 & 6 & 4 & 2 & 4 & 8 & 1 & 1 & 0 & 0 & 2 & 6 & 4 & 0 & 0 & 0 & 2\\
\hline
\textbf{\#5} & & 5 & 8 & 3 & 4 & 6 & 6 & 8 & 1 & 1 & 6 & 6 & 0 & 2 & 4 & 5 & 7 & 7 & 7 & 4 & 0 & 7 & 7 & 4 & 4\\
\hline
\textbf{\#6} & & 4 & 6 & 1 & 3 & 3 & 3 & 6 & 0 & 8 & 0 & 0 & 6 & 6 & 6 & 3 & 3 & 3 & 4 & 8 & 3 & 6 & 3 & 3 & 3\\
\hline
\textbf{\#7} & & 3 & 5 & 6 & 8 & 8 & 0 & 5 & 8 & 5 & 3 & 5 & 5 & 5 & 3 & 4 & 4 & 4 & 3 & 7 & 6 & 3 & 4 & 7 & 7\\
\hline
\textbf{\#8} & & 1 & 3 & 5 & 0 & 0 & 8 & 3 & 5 & 3 & 5 & 3 & 3 & 3 & 5 & 6 & 6 & 6 & 6 & 3 & 8 & 4 & 6 & 6 & 6\\
\hline
\textbf{\#9} & & 0 & 0 & 0 &